In [21]:
import cv2
import pyclipper
import numpy as np
from shapely.geometry import Polygon

In [38]:
canvas = np.zeros(shape=(500, 500, 3), dtype=np.uint8)
shrinker = pyclipper.PyclipperOffset()

In [39]:
polygon = (
    (348, 257), (364, 148), (362, 148), (326, 241),
    (295, 219), (258, 88), (440, 129), (370, 196), (372, 275)
)

offset = -7

shrinker.AddPath(polygon, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
shr_polygons = shrinker.Execute(offset)
print(type(shr_polygons[0]))

<class 'list'>


In [40]:
cv2.polylines(img=canvas, pts=[np.int32(polygon)], isClosed=True, color=(0, 0, 255), thickness=1)

for shr_polygon in shr_polygons:
    cv2.polylines(img=canvas, pts=[np.int32(shr_polygon)], isClosed=True, color=(0, 255, 0), thickness=1)

cv2.imwrite('canvas.png', canvas)

True

In [1]:
import cv2
import pyclipper
import numpy as np
from shapely.geometry import Polygon


def shrink_polygon(points, r: float = 0.4):
    shrinker = pyclipper.PyclipperOffset()    

    poly = Polygon(points)
    if not poly.is_valid:
        raise ValueError('must be valid polygon.')
    # d = poly.area * (1 - r ** 2) / poly.length
    d = poly.area * (1 - r) / poly.length

    points = [tuple(point) for point in points]
    shrinker.AddPath(points, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    polys = shrinker.Execute(-d)
    
    return polys


def unshrink_polygon(points, r: float = 0.4):
    shrinker = pyclipper.PyclipperOffset()

    poly = Polygon(points)
    if not poly.is_valid:
        raise ValueError('must be valid polygon.')

    d = poly.area * (1 + r) / poly.length

    points = [tuple(point) for point in points]
    shrinker.AddPath(points, pyclipper.JT_ROUND, pyclipper.ET_CLOSEDPOLYGON)
    polys = shrinker.Execute(d)

    return polys

In [2]:
polygon = (
    (348, 257), (364, 148), (362, 148), (326, 241),
    (295, 219), (258, 88), (440, 129), (370, 196), (372, 275)
)

polygon = (

In [3]:
shrink_polys = shrink_polygon(polygon, r=0.4)

canvas = np.zeros(shape=(500, 500, 3), dtype=np.uint8)
cv2.polylines(img=canvas, pts=[np.int32(polygon)], isClosed=True, color=(0, 0, 255), thickness=1)
cv2.polylines(img=canvas, pts=np.int32(shrink_polys), isClosed=True, color=(0, 255, 0), thickness=1)

cv2.imwrite('shrink_canvas.png', canvas)

True

In [14]:
shrink_polys

[[[417, 135],
  [371, 180],
  [375, 150],
  [375, 145],
  [372, 141],
  [369, 138],
  [364, 137],
  [362, 137],
  [357, 138],
  [354, 141],
  [352, 144],
  [321, 224],
  [305, 213],
  [274, 103]]]

In [4]:
unshrink_polys = []
for poly in shrink_polys:
    unshrink_polys.extend(unshrink_polygon(poly))

canvas = np.zeros(shape=(500, 500, 3), dtype=np.uint8)
cv2.polylines(img=canvas, pts=np.int32(shrink_polys), isClosed=True, color=(0, 0, 255), thickness=1)
cv2.polylines(img=canvas, pts=np.int32(unshrink_polys), isClosed=True, color=(0, 255, 0), thickness=1)

cv2.imwrite('unshrink_canvas.png', canvas)

True